<span type="title">输入输出设备</span> | <span type="update">2018-07-19</span> | <span type="version">1</span>

<span type="intro"><p type="card-text">本章主要介绍输入输出设备。</p></span>

# 输出输出接口的基本功能

## 输入输出接口结构

对于模型机而言，输入输出设备可以看作特殊的存储器，当需要输入时，PC通过地址总线到输入设备寻找数据，当控制总线发出“读”指令的时候，通过数据总线进行输入和处理。

![](w9p1.png)

如上图所示，I/O接口类似于存储器，但是功能却有重要的区别，其不仅包含了数据输入和输出的寄存器，还包含了状态、控制寄存器、中断控制逻辑，这些结构的目的是为了延缓外设读写和CPU处理数据之间的巨大的速度差距，避免外设连累CPU。当然，地址、数据、控制总线被用来将存储器和I/O接口与CPU连接起来。

## 输入输出接口功能

1. 数据缓冲
    - 解决CPU和外设之间的速度差距
2. 提供联络信息
    - 协调与同步数据交换过程
3. 信号与信息格式的转换
    - 模/数、数/模转换，串/并、并/串转换，电平转换
4. 设备选择
5. 中断管理
6. 可编程功能

简而言之，就是处理CPU和外设之间的**转换(信号格式)、缓冲(数据缓冲,中断)和交换(设备选择,协调握手)。**

## 直接传递数据

常见的简单外设使用并行数据接口进行连接。对于像单纯的LED数码管或者拨码开关这类设备，直接I/O接口通过8bit的并行数据输入输出和它们相连。当需要读写数据时，通过I/O的数据输入输出寄存器即可直接和这些硬件交互。

## 复杂设备的握手信号

但是对于像打印机这样的设备，则需要避免CPU过快发送指令导致指令来不及被打印机处理这类事情，因此需要一组信号，称之位Handshaking信号。

对于数据输出而言，首先接口要求打印机输出准备好信号，将数据从数据输出寄存器放置到并行数据输出线上，接着打印机初始化，返回输出回答信号，告诉接口正在读你发的数据，同时暂不要发下一个数据，之后从线上读入数据，当读取和处理完毕后将输出回答置为无效。

而接口检测到无效的输出回答，就可以继续重复这一过程，进行发送。输入也是类似。

# I/O接口的编址方式

在实现上，I/O接口内部包含一组称为I/O端口的寄存器，每个I/O端口都有自己的端口地址（或者叫做端口号），以便CPU访问。

CPU想要将一条指令传送给[41H]，那么，到底这个地址指代的是存储器还是I/O接口？

## 常见的编址方式

常见的端口编址方式与两种：

I/O端口和存储器分开编址
- I/O映像的I/O方式，I/O Mapped I/O
- x86体系结构采用该方式

I/O端口和存储器统一编址
- 存储器映像的I/O方式，Memory Mapped I/O
- ARM、MIPS、PowerPC等体系结构采用该方式


## I/O指令

`IN AC, PORT` 在x86上指的是将外设端口的内容输入到AL或者AX。

`OUT PROT, AC` 指的是将AL或AX的内容输出到外设端口。

如下是将 `OUT 21H, AL` 指令的执行过程，这里是分开编址，因此需要指定`M/IO=0` 这个信号意味着是IO而不是内存中的 `21H` 需要接受指令。此指令将AL寄存器内容输出到I/O对应端口的数据输出寄存器。

![](w9p2.png)

## 编址方式优缺点

对于统一编址而言，CPU面向的整个地址空间既包含接口，也包含存储器，这样在硬件上就很简单，不用额外的信号指定地址是寄存器还是接口。这种方式对于接口可以直接使用和访问存储器相同的控制逻辑，内部结构简单，同时指令功能丰富。

但是其缺点是，因为端口占用了部分存储器空间，因此在早期的x86上市难以忍受的。此外，因为x86的指令很长，并且复杂，所以，让功能简单的端口共用这些指令效率太低。因此，统一编址常见于指令简单的ARM，MIPS设备。

对于x86而言，分开编址虽然在硬件上提升了复杂性，但是可以重新设计指令，逻辑和层次都比较清晰，新指令短，速度快，同时不会影响储存器空间。

# 输入输出控制方式

## 程序控制

程序控制包括两种，其一为无条件传送，比如数码管和拨码开关。其假定外设都已经准备好，不查询外设的工作状态。其二为程序查询传送方式，CPU执行循环，不断查询外设状态，当期就绪的时候，才进行数据传送。

其方式如下所示：

对于数据输出过程：

![](w9p3.png)

- 首先CPU执行指令，将控制字写入 `3控制寄存器`，设置接口工作模式。
- CPU执行指令，将数据写入到 `1输出缓冲寄存器`
- 接口将数据发送到并行`数据输出信号线`，由接口或者CPU将 `输出准备好` 信号设置为有效。
- 外设发现 `输出准备好` 信号有效之后，从 `并行数据输出` 接受数据，并且将 `输出回答` 信号设置为有效。
- 等待外设处理数据自动将输出回答置为无效
- 当接口发现 `输出回答` 信号无效后，将 `2状态寄存器` 中的状态位 输出缓冲空 置为有效。
- CPU循环执行从 `2状态寄存器` 读出状态字，当发现 输出缓冲空 之后，继续下一个输出过程，输出新数据。

对于数据输入过程：

![](w9p4.png)

- 首先CPU将控制字写入 `3控制寄存器`，设置接口的工作模式
- 外设将 `数据` 发送到 `并行数据输入` 线上，并且将　`输入准备好` 信号设置为有效
- 接口发现 `输入准备好` 信号有效时，就从 `并行数据输入` 信号线接受数据，放置到 `输入缓冲寄存器` ，将 `输入回答` 信号设置为有效，阻止进一步输入。
- 接口更改 `状态寄存器` 的输入缓冲满状态位，将其置为有效
- CPU循环从 `状态寄存器`读状态字，直到发现输入缓冲满信号，执行指令从 `输入缓冲寄存器` 读数据
- 接口将 `输入回答` 信号设置为无效，等待外设输入新数据

**程序控制优缺点**：

对于无条件传送，其只适合控制程序简单的设备，对于程序查询传送方式，其比无条件传送方式更加可靠，但是查询外设占用了大量的时间。其共同的特点是，对外设的要求低，操作流程清晰，但是由于CPU循环在任意一次寄存器满之后都要等待状态寄存字，自身用来传送数据，因此占用了宝贵的计算资源。

## 中断控制方式

中断控制方式的写入过程如下：

- CPU 通过 控制寄存器 设置接口工作模式。
- 外设发送`数据`到`并行输入`，同时发送`输入准备好`信号
- 接口将数据搬移到 `输入缓冲寄存器`，并且将 `输入回答` 设置为有效
- 现在CPU不再轮询状态寄存器，因此接口通过 `中断控制逻辑` 向CPU发送中断请求，并且将 `状态寄存器` 中的输入缓冲满 设置为有效
- CPU进入中断服务程序，从状态寄存器读出状态字，发现输入缓冲满指令后，从输入缓冲寄存器读出数据
- 接口将输入回答信号设置为无效，等待进一步输入

![](w9p5.png)

中断控制方式的数据输出也是类似，中断只是为了避免在外设准备输出的时候降低CPU的损耗， 在传统方式中，CPU发送指令后一直等待状态寄存器中的状态字，然后进一步输出。而状态字的改变则需要外设的输出回答。

现在CPU不等待输出回答，如果有输出回答，接口通过中断告诉CPU，CPU进入中断后从状态寄存器查询状态字，然后开始下一个输出过程，继续输出新数据。

中断控制程序可以让CPU和外设并行工作，可以说起到了很好的速度缓冲，外设申请服务主动权，在一定程度上满足和I/O处理实时性的要求，但是外设和cun'chu